# LLM & RAG를 활용한 AI 서비스 만들기 - 5주차

### 5.2 Vector DB 개념 및 RAG (Retrieval-Augmented Generation) 개념

In [3]:
import torch
print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

2.5.1+cu124


device(type='cuda')

#### 한국어 데이터 임베딩 실습

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Multilingual-E5-large-instruct 모델 로드
model = SentenceTransformer('intfloat/multilingual-e5-large').to(device)

# 문장 리스트
sentences = [
    "참새는 짹짹하고 웁니다.",
    "LangChain과 Faiss를 활용한 예시입니다.",
    "자연어 처리를 위한 임베딩 모델 사용법을 배워봅시다.",
    "유사한 문장을 검색하는 방법을 살펴보겠습니다.",
    "강좌를 수강하시는 수강생 여러분 감사합니다!"
]

# 문장들을 임베딩으로 변환
embeddings = model.encode(sentences)

# 임베딩 벡터 출력
print(embeddings.shape)  # 4개의 문장이 1024 차원의 벡터로 변환됨
print(embeddings[0])

d:\Anaconda3_envs\Sparta\main_camp\ch3_RAG\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\Anaconda3_envs\Sparta\main_camp\ch3_RAG\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--intfloat--multilingual-e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to r

(5, 1024)
[ 0.03305342  0.00946585 -0.00015371 ... -0.02273227  0.00190389
  0.01841401]


### 5.5 Python LangChain과 FAISS

In [17]:
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# .env 파일에서 OPENAI API KEY 가져오기
api_key = os.getenv("OPENAI_API_KEY")

#### LangChain 기본 개념

##### 언어 모델 초기화

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4")

# 모델에 메시지 전달
response = model.invoke([HumanMessage(content="안녕하세요, 무엇을 도와드릴까요?")])
print(response.content)

사용자의 요청에 따라 다양한 정보 제공, 일정 관리, 알람 설정, 뉴스 업데이트, 음악 재생 등을 도와드릴 수 있습니다. 구체적으로 어떤 도움이 필요한지 알려주시면 감사하겠습니다.


##### 프롬프트 템플릿 사용하기

In [19]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 메시지 설정
system_template = "Translate the following sentence from English to {language}:"

# 사용자 텍스트 입력
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "{text}")
])

# 프롬프트 생성
result = prompt_template.invoke({"language": "French", "text": "How are you?"})
print(result.to_messages())

[SystemMessage(content='Translate the following sentence from English to French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})]


#### LangChain Expression Language (LCEL)로 체인 연결

In [20]:
from langchain_core.output_parsers import StrOutputParser

# 응답을 파싱하는 파서 초기화
parser = StrOutputParser()

# 템플릿, 모델, 파서를 체인으로 연결
chain = prompt_template | model | parser

# 체인 실행
response = chain.invoke({"language": "Korean", "text": "Where is the library?"})
print(response)

도서관은 어디에 있나요?


#### FAISS를 활용한 벡터 데이터베이스 구성 및 쿼리

##### Step 1: OpenAI 임베딩 모델로 벡터 임베딩 생성

In [21]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

##### Step 2: FAISS 인덱스 초기화

In [22]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# FAISS 인덱스 생성
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

##### Step 3: 벡터 데이터베이스에 문서 추가

In [ ]:
from langchain_core.documents import Document
from uuid import uuid4  # 고유 ID를 생성해주는 라이브러리

# 문서 생성
documents = [
    Document(page_content="LangChain을 사용해 프로젝트를 구축하고 있습니다!", metadata={"source":"tweet"}),
    Document(page_content="내일 날씨는 맑고 따뜻할 예정입니다.", metadata={"source":"news"}),
    Document(page_content="오늘 아침에는 팬케이크와 계란을 먹었어요.", metadata={"source":"personal"}),
    Document(page_content="주식 시장이 경기 침체 우려로 하락 중입니다.", metadata={"source":"news"}),
]

# 고유 ID 생성 및 문서 추가
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['4068b581-3ca8-4d84-8327-05481b4ae96f',
 'e3973e77-b0f7-42b1-8a1a-8320929475ae',
 'd8d7cc6c-0975-46bd-af7a-d62e59ef4f1c',
 'bc5e69d5-355b-4b00-a532-c15ea053d6c1']

##### Step 4: 벡터 데이터베이스 쿼리

In [24]:
# 기본 유사성 검색
results = vector_store.similarity_search("내일 날씨는 어떨까요?", k=2, filter={"source": "news"})
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")
    
# 점수와 함께 유사성 검색
results_with_scores = vector_store.similarity_search_with_score("LangChain에 대해 이야기해주세요.", k=2, filter={"source": "tweet"})
for res, score in results_with_scores:
    print(f"* [SIM={score:.3f}] {res.page_content} [{res.metadata}]")

* 내일 날씨는 맑고 따뜻할 예정입니다. [{'source': 'news'}]
* 주식 시장이 경기 침체 우려로 하락 중입니다. [{'source': 'news'}]
* [SIM=0.159] LangChain을 사용해 프로젝트를 구축하고 있습니다! [{'source': 'tweet'}]


#### RAG 체인에 FAISS 통합

##### Step 1: Retriever로 변환

In [25]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

##### Step 2: RAG 체인 생성

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
    
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}
    
# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,  # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()  # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
} | DebugPassThrough() | ContextToText() | contextual_prompt | model

# 질문 실행 및 각 단계 출력 확인
response = rag_chain_debug.invoke("강사 이름은?")
print("Final Response:")
print(response.content)

Debug Output: 강사 이름은?
Debug Output: {'context': [Document(metadata={'source': 'tweet'}, page_content='LangChain을 사용해 프로젝트를 구축하고 있습니다!')], 'question': '강사 이름은?'}
Final Response:
The context does not provide information on the instructor's name.


#### FAISS 인덱스의 저장 및 로드

In [28]:
# 인덱스 저장
vector_store.save_local("faiss_index")

# 저장된 인덱스 로드
new_vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)


#### FAISS 데이터베이스 병합

In [29]:
db1 = FAISS.from_texts(["문서 1 내용"], embeddings)
db2 = FAISS.from_texts(["문서 2 내용"], embeddings)

# 병합
db1.merge_from(db2)
